# Artificial Neural Networks

### Importing the libraries

In [1]:
import numpy as np
# don't need matplotlib
import pandas as pd
import tensorflow as tf

2023-10-15 17:56:23.828410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-15 17:56:23.828472: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-15 17:56:23.828498: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-15 17:56:23.835419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 17:56:24.681960: W tensorflow/compiler/

In [2]:
tf.sysconfig.get_build_info() 

OrderedDict([('cpu_compiler', '/usr/lib/llvm-16/bin/clang'),
             ('cuda_compute_capabilities',
              ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '11.8'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [3]:
tf.__version__

'2.14.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [4]:
dataset = pd.read_csv("Churn_Modelling.csv")

# independent variables (excluding the first three columns: entry number, customer ID, customer name)
# customer name is truly irrelevant to the model?
X = dataset.iloc[:, 3:-1].values
# dependent variable (variable to be predicted; rightmost column)
y = dataset.iloc[:, -1].values

In [5]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding Categorical Data

#### Label encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
# encoding just the gender column

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the data into the training set and test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Feature Scaling

In [11]:
# Feature scaling is absolutely compulsory for all data in deep learning, even binary variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

## Part 2: Building the ANN

### Initializing the ANN

In [12]:
# Keras used to be its own library but was integrated into TensorFlow as of 2.0
ann = tf.keras.models.Sequential()

2023-10-15 17:57:27.037861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-15 17:57:27.110972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-15 17:57:27.111032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-15 17:57:27.112669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-15 17:57:27.112730: I tensorflow/compile

### Adding the input layer and the first hidden layer

In [13]:
# there is no perfect answer for the best number of neurons in the hidden layer. It requires experimentation.
ann.add(tf.keras.layers.Dense(units=6, activation="relu")) # relu = rectifier

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid")) # sigmoid returns a probability of being either 0 or 1
# The outcome is binary, so one neuron is enough. For classification tasks you would need more


## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# adam = a sochastic gradient descent optimizer
# binary cross entropy is always used for binary dependent variables
# you can enter many metrics, but we are ony using one

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size= 32, epochs=100) # 32 is the default batch size value

Epoch 1/100


2023-10-15 17:57:56.082051: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f282f8eb4e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-15 17:57:56.082095: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 980 Ti, Compute Capability 5.2
2023-10-15 17:57:56.096984: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-15 17:57:58.377156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2023-10-15 17:57:58.477412: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


250/250 [==============================] - 7s 6ms/step - loss: 0.5518 - accuracy: 0.7856
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4661 - accuracy: 0.7994
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4409 - accuracy: 0.8121
Epoch 4/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4262 - accuracy: 0.8211
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4158 - accuracy: 0.8291
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4065 - accuracy: 0.8340
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3956 - accuracy: 0.8391
Epoch 8/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3824 - accuracy: 0.8472
Epoch 9/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3696 - accuracy: 0.8529
Epoch 10/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3609 - accuracy: 0.8545
E

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Homework: Use the ANN model to predict if the customer with the following information will leave the bank:

Geography: France  
Credit score: 600  
Gender: Male  
Age: 40  
Tenure: 3 years  
Balance: 60000  
Number of products: 2  
Does this customer have a credit card? Yes  
Is this customer an active member: yes  
Estimated Salary: $50000  
So, should we say goodbye to the customer?

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) # converts output above 0.5 to true
# Must have the right number of elements!

1/1 [==============================] - 0s 98ms/step
[[False]]


### Predicting the Test set results

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1 )), 1)) 

63/63 [==============================] - 0s 3ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 0]]


### Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

[[1528   57]
 [ 207  208]]


0.868